In [1]:
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
import numpy as np

from qutip import * 
import os

# Matplotlib Einstellungen gemäß den LaTeX-Caption-Formatierungen
plt.rcParams.update({
#    'text.usetex': True,              # Enable LaTeX for text rendering
#    'font.family': 'serif',           # Use a serif font family
#    'font.serif': 'Palatino',         # Set Palatino as the serif font
#    'text.latex.preamble': r'\usepackage{amsmath}',
#    'font.size': 20,                   # Font size for general text
#    'axes.titlesize': 20,              # Font size for axis titles
#    'axes.labelsize': 20,              # Font size for axis labels
#    'xtick.labelsize': 20,             # Font size for x-axis tick labels
#    'ytick.labelsize': 20,             # Font size for y-axis tick labels
#    'legend.fontsize': 20,             # Font size for legends
#    'figure.figsize': [8, 6],          # Size of the plot (width x height)
#    'figure.autolayout': True,         # Automatic layout adjustment
#    'savefig.format': 'svg',           # Default format for saving figures
#    'figure.facecolor': 'none',        # Make the figure face color transparent
#    'axes.facecolor': 'none',          # Make the axes face color transparent
#    'savefig.transparent': True        # Save figures with transparent background
})
output_dir = os.getcwd()  # Gets the current working directory
os.makedirs(output_dir, exist_ok=True)

# allows for interactive plots
#%matplotlib notebook

$$
H = H_0 + H_I
$$

$$
H_0 = \hbar \omega_a \sum_{i=1}^{N_{\text{atoms}}} \sigma_+^{(i)} \sigma_-^{(i)} + \hbar \sum_{i,j=1}^{N_\text{{atoms}}} J_{ij} \sigma_+^{(i)} \sigma_-^{(j)} \quad \text{,} \quad J_{ij} = \frac{\alpha}{|r_i - r_j|^3}
$$

$$
H_I = \hbar D \cdot E(t)
\quad \text{,} \quad D = \mu \sum_{i=1}^{N_{\text{atoms}}} \sigma_+^{(i)} + \sigma_-^{(i)}
\quad \text{,} \quad E(t_i) \propto \Omega_{coupling} \cos(\pi (t - t_i)) \delta(t_i)
$$

### Decay operators single case
$$
C_{\text{decay}}^{(i)} = \sqrt{\gamma_0} \sigma_-^{(i)} \quad
C_{\text{dephase}}^{(i)} = \sqrt{\gamma_\phi} \sigma_z^{(i)}
$$

In [17]:
#
# Set the system parameters
#
# ENERGY LANDSCAPE, c = 1, hbar = 1
fixed_lam = 1. #
alpha     = 1. # coupling strength of the dipoles       Fine structure const?

omega_a   = 2 * np.pi / fixed_lam   # energysplitting of the atom, when ground state is set to 0
mu        = 1 * omega_a             # Dipole matrix element of each atom
omega_R   = 1 * omega_a             # Rabi freq coupling to laser field  for first 2 lasers  -> E_field_0 dot dot Dip_op, parallel

# LINBLAD OPS
gamma_0   = .1 # decay rate of the atoms
gamma_phi = .1 # dephasing rate of the atoms

# TOPOLOGY
n_chains = 1 # number of chains
n_rings  = 1 # number of rings
N_atoms  = n_chains * n_rings  # number of atoms

distance = 1. # * fixed_lam # defining topology

# TIME EVOLUTION
t_max = 10  # Maximum time, replace with the required value
fine_steps = 100  # Number of steps for high-resolution
medium_steps = fine_steps//1  # Number of steps for medium resolution
sparse_steps = medium_steps//10  # Number of steps for medium resolution

# High-resolution time array
times = np.linspace(0, t_max, fine_steps)

# Sparse time array
times_T = np.linspace(0, t_max, sparse_steps)
mean_pos = [0, 0, 0]  # Mean position (can be any point in 3D space)
sigma_pos = N_atoms / 10  # Standard deviation for position distribution
########################################               define the geometry                 #############################################
def chain_positions(distance, N_atoms):
    Pos = np.zeros((N_atoms, 3))
    for i in range(N_atoms):
        Pos[i, 2] = i * distance
    return Pos
def z_rotation(angle):
    return np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle),  np.cos(angle), 0],
        [0,              0,            1]])
def ring_positions(distance, n_chains):
    Pos = np.zeros((n_chains, 3))
    dphi = 2 * np.pi / n_chains
    if n_chains == 1:
        radius = 0
    else:
        radius = distance / 2 / np.sin(np.pi / n_chains)
    helper = np.array([radius, 0, 0])
    for i in range(n_chains):
        rotation_matrix = z_rotation(dphi * i)
        Pos[i] = np.matmul(rotation_matrix, helper)
    return Pos
def cyl_positions(distance, N_atoms, n_chains):
    Pos = np.zeros((N_atoms, 3))
    Pos_chain = chain_positions(distance, N_atoms // n_chains)
    Pos_ring = ring_positions(distance, n_chains)
    for i in range(n_chains):
        Pos[i * (N_atoms // n_chains): (i + 1) * (N_atoms // n_chains)] = Pos_chain + Pos_ring[i]
    return Pos
##################                           help functions                                 ##################
def sample_positions(mean_pos, sigma_pos, N_atoms):
    positions = []
    while len(positions) < N_atoms:
        # Sample a new position for the atom
        new_position = np.random.normal(loc=mean_pos, scale=sigma_pos, size=3)
        # Check if the new position is unique (no duplicates)
        if not any(np.allclose(new_position, pos) for pos in positions):
            positions.append(new_position)
    return np.array(positions)
def sample_frequencies(E0, Delta, N_atoms):
    # Sample N_atoms frequencies from the Gaussian distribution
    frequencies = np.random.normal(loc=E0, scale=Delta/2, size=N_atoms)
    return frequencies

def plot_positive_color_map(x, y, data, T = np.inf, space="real", type="real", positive=False, safe=False, output_dir=None, fixed_lam=None, alpha=None, gamma_0=None, gamma_phi=None, n_rings=None, n_chains=None, distance=None):
    """
    Create a color plot of 2D functional data for positive x and y values only.

    Parameters:
        x (np.array): 1D array of x values.
        y (np.array): 1D array of y values.
        data (np.array): 2D array of data values aligned with x and y.
        T (float): Temperature parameter to include in plot title and file name.
        space (str): Either 'real' or 'freq' specifying the space of the data.
        type (str): Type of data ('real', 'imag', 'abs', or 'phase'). Used only if space="freq".
        positive (bool): Whether to use only positive values of x and y.
        safe (bool): If True, saves the plot to a file.

    Returns:
        None
    """
    # Convert x and y into 1D arrays if they're not
    x = np.array(x)
    y = np.array(y)

    if positive:
        # Filter for positive x and y values
        positive_x_indices = np.where(x > 0)[0]  # Indices where x > 0
        positive_y_indices = np.where(y > 0)[0]  # Indices where y > 0
        x = x[positive_x_indices]
        y = y[positive_y_indices]
        data = data[np.ix_(positive_y_indices, positive_x_indices)]

    if space == "real":
        colormap = "viridis"
        label = r"$E_{out} \propto P / E_0$"
        title = f"Real space 2D Spectrum (arb. units)"
        if T != np.inf:
            title += f" at T ={T:.2f}"
        x_title = r"$t_{det}$ (arb. units)"
        y_title = r"$\tau_{coh}$ (arb. units)"
    elif space == "freq":
        if type == "real":
            title = f"Freq space, Real 2D Spectrum (arb. units)"
            data = np.real(data)
        elif type == "imag":
            title = f"Freq space, Imag 2D Spectrum (arb. units)"
            data = np.imag(data)
        elif type == "abs":
            title = f"Freq space, Abs 2D Spectrum (arb. units)"
            data = np.abs(data)
        elif type == "phase":
            title = "Freq space, Phase 2D Spectrum (arb. units)"
            data = np.angle(data)
        else:
            raise ValueError("Invalid Type. Must be 'real', 'imag', 'abs', or 'phase'.")

        colormap = "plasma"
        label = r"$E_{out} \propto P / E_0$"
        if T != np.inf:
            title += f" at T ={T:.2f}"

        x_title = r"$\omega_{t_{det}}$ (arb. units)"
        y_title = r"$\omega_{\tau_{coh}}$ (arb. units)"

    else:
        raise ValueError("Invalid space. Must be 'real' or 'freq'.")

    # Check and adjust the dimensions of x, y, and data
    if data.shape[1] != len(x):
        raise ValueError(f"Length of x ({len(x)}) must match the number of columns in data ({data.shape[1]}).")
    if data.shape[0] != len(y):
        raise ValueError(f"Length of y ({len(y)}) must match the number of rows in data ({data.shape[0]}).")

    # Add 1 to the dimensions of x and y for correct pcolormesh behavior
    x = np.concatenate([x, [x[-1] + (x[-1] - x[-2])]])  # Add an extra value for the last x edge
    y = np.concatenate([y, [y[-1] + (y[-1] - y[-2])]])  # Add an extra value for the last y edge

    # Plot the color map
    plt.figure(figsize=(8, 6))
    plt.pcolormesh(x, y, data / omega_R, shading="auto", cmap=colormap)
    plt.colorbar(label=label)
    plt.title(title)
    plt.xlabel(x_title)
    plt.ylabel(y_title)

    # Save the plot if safe is True
    if safe and output_dir is not None:
        file_name_combined = (
            f"Classical_lam{fixed_lam:.1f}_alpha={alpha:.2f}_g_0{gamma_0:.2f}_g_phi{gamma_phi:.2f}_"
            f"{n_rings}x{n_chains}_dist={distance:.2f}_positive={positive}_space={space}"
        )
        if space == "freq":
            file_name_combined += f"_type={type}"
        file_name_combined += ".svg"
        save_path_combined = os.path.join(output_dir, file_name_combined)
        plt.savefig(save_path_combined)

    plt.show()
def Hamilton0(distance, n_rings, n_chains):
    N_atoms = n_chains * n_rings
    Pos = cyl_positions(distance, N_atoms, n_chains) # sample_positions(mean_pos, sigma_pos, N_atoms)        #
    atom_frequencies = sample_frequencies(omega_a, 0.0125 * omega_a, N_atoms) # [omega_a] * N_atoms #
    H = 0
    for a in range(N_atoms):
        for b in range(N_atoms):
            op = sm_list[a].dag() * sm_list[b]
            if a != b:
                ra, rb = Pos[a, :], Pos[b, :]
                H += alpha / (np.linalg.norm(rb-ra))**3 * op
            else:
                H += atom_frequencies[a] *op # Diagonals except for |0><0|
    return H
def El_field(t, args):
    t0 = args['time']
    Delta = args['Delta']
    E = np.exp(1j*(args['omega'] * t + args['phi']))
    E += np.conjugate(E)
    # secure the field is 0 outside short range
    E *= np.cos(np.pi * (t - t0) / (2 * Delta)) ** 2 * np.heaviside(t - (t0 - Delta), 0) * np.heaviside(t0 + Delta - t,0)
    return args['E0'] * E

# Define the ground & the excited states
# atomic dofs
atom_g = basis(N_atoms + 1, 0)
atom_es = [basis(N_atoms + 1, i) for i in range(1, N_atoms + 1)]
# initial state
psiini = atom_g # = |g>_atom

# combined dofs
sm_list = []    # lowering operators of atomic system
Dip_op = 0         # collective sigma_x operator for the system
for i in range(N_atoms):
    op = atom_g * atom_es[i].dag()
    sm_list.append(op)
    Dip_op += mu * op + mu * op.dag()
H0 = Hamilton0(distance, n_rings, n_chains)

# Jump / Expect Operators          # Define the decay collapse and dephasing operator for each spin
# Collapse operators
c_op2 = [np.sqrt(gamma_0) * op for op in sm_list]                         # Individual atom decays
c_op4 = [np.sqrt(gamma_phi) * commutator(op.dag(), op) for op in sm_list] # Individual atom dephasing
c_op_list = c_op2 + c_op4  # Combine all collapse operators
# Expectation operators for measuring populations across atomic ground and excited levels
e_op_list = [ket2dm(basis(N_atoms + 1, i)) for i in range(N_atoms + 1)]

#
# evolution with t
#
# create the time dependant evolution
Omegas = [omega_R, omega_R, omega_R/10] # Probe pulse is smaller
omegas = [omega_a, omega_a, omega_a]    # The laser is on resonant
Delta_ts = [t_max*1e-3, t_max*1e-3, t_max*1e-3] # narrow width of the pulses
HI = [-Dip_op, El_field] # interaction Hamiltonian with function-based time dependence
H = [H0, HI]
options = Options(store_states=True)


#
# PREPROCESSING
#
# Phase cycling
phases = [-1 * i * np.pi / 2 for i in range(4)]

In [18]:
def get_times_for_T(T, steps=medium_steps):
    """
    Generate medium-sparse time arrays for a given T.

    Parameters:
        T (float): The value of T.
        t_max (float): Maximum time value.
        steps (int): Number of steps for medium resolution.

    Returns:
        tuple: Arrays (times_t, times_tau)
            times_t (ndarray): Array from T to t_max with medium resolution.
            times_tau (ndarray): Array from 0 to T with medium resolution.
    """
    times_t = np.linspace(0, t_max - T - 2 * Delta_ts[2], steps)
    times_tau = np.linspace(0, t_max - T - 2 * Delta_ts[0], steps)
    return times_t, times_tau

def process_close_times(fine_times, times_tau, tolerance=1e-2):
    """
    Processes fine times and returns indices of fine_times when a time is close to a value in times_tau.
    Ensures each tau is handled only once.

    Parameters:
        fine_times (list or array): High-resolution time values.
        times_tau (list or array): Target time values to match against.
        tolerance (float): Maximum allowed difference to consider two times "close."

    Returns:
        List[int]: Indices of fine_times where the times are close to values in times_tau.
    """
    # Keep track of already processed tau values
    processed_taus = set()
    close_indices = []

    # Iterate over the fine times array
    for i, time in enumerate(fine_times):
        # Check for close times_tau values
        for tau in times_tau:
            if tau not in processed_taus and abs(time - tau) <= tolerance:
                # Add the index to the list of close indices
                close_indices.append(i)

                # Mark this tau as processed to avoid duplicates
                processed_taus.add(tau)

    return close_indices

times_t_test, times_tau_test = get_times_for_T(0)
len(times), len(times_t_test), len(times_tau_test), len(times_T)
#close_times_indices = process_close_times(times, times_tau_test)
#print(close_times_indices, times_tau_test, times[close_times_indices])

(100, 100, 100, 10)

In [19]:
#
# CALCULATIONS
#
def compute_pulse(psiini, times, phi, i):
    args = {
        'phi': phi,
        'time': Delta_ts[i],
        'omega': omegas[i],
        'Delta': Delta_ts[i],
        'E0': Omegas[i]
    }
    result = mesolve(H, psiini, times, c_ops=c_op_list, e_ops=e_op_list, args=args, options=options)
    return result.states
def compute_two_dimensional_polarization(T_val, phi_1, phi_2):
    times_0 = times
    t_values, tau_values = get_times_for_T(T_val)
    data = np.zeros((len(tau_values), len(t_values))) # x -> t, y -> tau
    close_times_tau = process_close_times(times_0 - Delta_ts[0], tau_values)

    data_1 = compute_pulse(psiini, times_0, phi_1, 0) # first pulse

    for tau_idx, tau in enumerate(tau_values):
        i = np.abs(tau_values[tau_idx] - (times_0 + Delta_ts[1] - Delta_ts[0])).argmin()  # find the index where T_val is
        psi_1 = data_1[i]  # Access psi_2 after waiting T

        times_1 = times[i:] - times[i]

        data_2 = compute_pulse(psi_1, times_1, phi_2, 1) # second pulse

        j = np.abs(times_1 + Delta_ts[2] - Delta_ts[1] - T_val).argmin()  # find the index where T_val is

        T_curr = times_1[j] + Delta_ts[2] - Delta_ts[1]  # current time val

        psi_2 = data_2[j]  # Access psi_2 after waiting T
        times_2 = times[j:] - times[j]

        close_times_t = process_close_times(times_2 - Delta_ts[2], t_values)

        data_f = compute_pulse(psi_2, times_2, 0, 2) # third pulse

        for t_idx, t in enumerate(t_values):
            k = np.abs(t_values[t_idx] - times_2 - Delta_ts[2]).argmin()  # find the closest index
            psi_f = data_f[k]  # Access psi_2 after waiting T
            # Compute the polarization and store it in the dictionary
            Polarization = expect(Dip_op, psi_f)
            data[tau_idx, t_idx] = Polarization

    # Crop non-zero data
    non_zero_rows = np.any(data != 0, axis=1)
    non_zero_cols = np.any(data != 0, axis=0)
    data = data[non_zero_rows][:, non_zero_cols]
    tau_values = tau_values[non_zero_rows]
    t_values = t_values[non_zero_cols]

    return tau_values, t_values, data
def process_phi_combination(phi_1, phi_2):
    dict_T = {}
    for T in times_T:
        tau_values, t_values, data = compute_two_dimensional_polarization(T, phi_1, phi_2)
        dict_T[T] = tau_values, t_values, data
    return dict_T
x = compute_two_dimensional_polarization(3, 0, 0)
plot_positive_color_map(x[1], x[0], x[2], safe=False)

KeyboardInterrupt: 

In [20]:
# Initialize results dictionary
all_results = {}
# Parallelize the computation for all combinations of phi_1 and phi_2
max_workers = min(len(phases) ** 2, os.cpu_count())  # Limit workers based on available CPUs
with ProcessPoolExecutor(max_workers=max_workers) as executor:
    # Track each future and corresponding (phi_1, phi_2)
    futures = {executor.submit(process_phi_combination, phi_1, phi_2): (phi_1, phi_2)
               for phi_1 in phases for phi_2 in phases}

    # Collect results as they complete
    for future in futures:
        phi_1, phi_2 = futures[future]  # Retrieve the associated (phi_1, phi_2)
        third_pulse_data = future.result()
        if third_pulse_data is not None:
            all_results[(phi_1, phi_2)] = third_pulse_data

#all_results[(0,0)] # now contains the results keyed by (phi_1, phi_2)

In [30]:
for T_j, (taus, ts, data_time) in T_averaged_dict.items():  # Use enumerate to count iterations
    if T_j == 0:  # First iteration
        global_ts = ts
        global_taus = taus
        global_t_freqs = (np.fft.fftfreq(len(global_ts), d=(global_ts[1] - global_ts[0])))  # Frequency axis for detection time
        global_tau_freqs = (np.fft.fftfreq(len(global_taus), d=(global_taus[1] - global_taus[0])))  # Frequency axis for excitation time

        # Sum all data into global sum
        global_data_time = np.zeros((len(global_taus), len(global_ts)))
        global_data_freq = np.zeros((len(global_taus), len(global_ts)), dtype=np.complex64)


    if len(taus) < 4 or len(ts) < 4:
        print(f"Skipped T_j={T_j} due to insufficient data (taus={len(taus)}, ts={len(ts)})")
        continue

    # Prepare to extend local data into global coordinates
    data_extended_time = np.zeros_like(global_data_time)

    # Map taus and ts to global indices
    global_tau_indices = {tau: idx for idx, tau in enumerate(global_taus)}
    global_t_indices = {t: idx for idx, t in enumerate(global_ts)}

    # Map taus and ts to global indices
    for i, tau in enumerate(taus):
        global_i = np.where(global_taus == tau)[0]  # Array of indices where condition is True
        if global_i.size > 0:  # Ensure we have valid matches
            global_i = global_i[0]
        else:
            continue  # Skip if no valid mapping

        for j, t in enumerate(ts):
            global_j = np.where(global_ts == t)[0]  # Array of indices where condition is True
            if global_j.size > 0:  # Ensure we have valid matches
                global_j = global_j[0]
            else:
                continue  # Skip if no valid mapping

            # Assign values if valid global indices exist
            data_extended_time[global_i, global_j] = data_time[i, j]
            global_data_time[global_i, global_j] += data_time[i, j]

    data_extended_freq = np.fft.fft2(data_extended_time)
    plot_positive_color_map(ts, taus, data_time, T_j)
    plot_positive_color_map(np.fft.fftfreq(len(ts), d=(ts[1] - ts[0])), np.fft.fftfreq(len(taus), d=(taus[1] - taus[0])), np.fft.fft2(data_time), T_j, space = "freq", type = "real", positive = True, safe = False)

#    plot_positive_color_map(global_ts, global_taus, data_extended_time, T_j, safe = False)
#    plot_positive_color_map(global_t_freqs, global_tau_freqs, data_extended_freq, T_j, space = "freq", type = "real", positive = True, safe = False)
    global_data_freq += data_extended_freq

# Plot the aggregated results
#plot_positive_color_map(global_ts, global_taus, global_data_time, safe=False)  # Global time-space data
#plot_positive_color_map(global_t_freqs, global_tau_freqs, (global_data_freq), space="freq", type="real", positive=True, safe=False)  # Global frequency-space data